In [ ]:
import keras
import tensorflow as tf
keras.__version__
from scipy.io import loadmat
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras import optimizers, layers, models, regularizers, losses, metrics

In [ ]:
data = np.genfromtxt('TrackTrainingSetArSc.csv',delimiter=' ', skip_header=1)
print(("Rozmiar zestawu treningowego = %d x %d ")%(data.shape[0], data.shape[1]))
datapd = pd.read_csv('TrackTrainingSetArSc.csv', sep=' ', header=0)
datapd = datapd.drop(['qOverPXZ'], axis=1)
datapd.head()

In [ ]:
jednostki = {'p': ' [GeV]', 
             'px': ' [GeV]', 
             'py': ' [GeV]', 
             'pz': ' [GeV]',
             'enDep': ' [GeV]',
             'pFirstX': ' [cm]',
             'pFirstY': ' [cm]',
             'pFirstZ': ' [cm]',
             'pLastX': ' [cm]',
             'pLastY': ' [cm]',
             'pLastZ': ' [cm]',
             'pFirstBx': ' [T]',
             'pFirstBy': ' [T]',
             'pFirstBz': ' [T]',
             'pLastBx': ' [T]',
             'pLastBy': ' [T]',
             'pLastBz': ' [T]',
             'q': '', 
             'Ndedx': '',
             'qOverPXZ': '',
             'nMaxPoint': ''}
print(datapd.keys())

In [ ]:
fsize = 20
i=1
fig = plt.figure(figsize=(25,25))
for key in datapd.keys():
    plt.subplot(8,3,i)
    i+=1
    plt.hist(datapd[key], bins=40)
    plt.xlabel(str(key) + jednostki[key], fontsize=fsize*1.1) #fontsize zeby zwiekszyc etykiete
    plt.tight_layout()
    plt.xticks(size=fsize)
    plt.yticks(size=fsize)
    plt.yscale('log')
plt.grid()
plt.show()
fig.savefig('RozkladCech.png')

In [ ]:
i=1
fig = plt.figure(figsize=(14,12))
for key in datapd.drop("nMaxPoint", axis=1).keys():
    plt.subplot(5,4,i)
    i+=1
    plt.hist2d(x=datapd[key], y=datapd['nMaxPoint'], bins=50, cmap='jet', norm=LogNorm())
    plt.xlabel(str(key) + jednostki[key], fontsize=fsize*0.6)
    plt.ylabel('nMaxPoint', fontsize=fsize*0.6)
    plt.colorbar()
    plt.tight_layout()
    plt.xticks(size=fsize*0.5)
    plt.yticks(size=fsize*0.5)
plt.grid()
plt.show()
fig.savefig('ZaleznosciNMaxPointsOdCech.png')

In [ ]:
X, y = np.array(datapd.drop(['nMaxPoint'], axis=1)), np.array(datapd['nMaxPoint'])

X = X[:,3:19] 

y_lin = y.astype('float32')
y_log = to_categorical(y, num_classes = 241)


train, test, train_label_lin, test_label_lin = train_test_split(X, y_lin, shuffle=True, test_size=0.25)
train, test, train_label_log, test_label_log = train_test_split(X, y_log, shuffle=True, test_size=0.25)

In [ ]:
mean = np.mean(X, axis = 0)
std = np.std(X, axis = 0)


train_normalized = (train - mean)/std
test_normalized = (test - mean)/std

In [ ]:
learning_rate = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                  factor=0.1, 
                                                  patience=15, 
                                                  verbose=1, 
                                                  mode='auto', 
                                                  min_delta=0.01, 
                                                  cooldown=0, 
                                                  min_lr=0)

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                           min_delta=0.001, 
                                           patience=20, 
                                           verbose=1, 
                                           mode='auto', 
                                           baseline=None, 
                                           restore_best_weights=True)


In [ ]:
drop = 0.1
model_lin = models.Sequential()
model_lin.add(layers.Dense(256, activation='relu', input_shape=(train.shape[1], ), kernel_initializer='random_normal'))
model_lin.add(layers.Dropout(drop))
model_lin.add(layers.Dense(256, activation='relu'))
model_lin.add(layers.Dropout(drop))
model_lin.add(layers.Dense(256, activation='relu'))
model_lin.add(layers.Dense(256, activation='relu'))
model_lin.add(layers.Dropout(drop))
model_lin.add(layers.Dense(256, activation='relu'))
model_lin.add(layers.Dense(256, activation='relu'))
model_lin.add(layers.Dense(256, activation='relu'))
model_lin.add(layers.Dense(256, activation='relu'))
model_lin.add(layers.Dense(1))

model_lin.compile(loss=losses.mean_squared_error, 
               optimizer=optimizers.Adam(),
               metrics=['mae'])

In [ ]:
model_lin.summary()

In [ ]:
history_lin = model_lin.fit(train_normalized, 
                      train_label_lin, 
                      epochs=200, 
                      validation_split=0.2, 
                      batch_size=64, 
                      callbacks=[learning_rate, early_stop])

In [ ]:
drop = 0.1
model_log = models.Sequential()
model_log.add(layers.Dense(256, activation='relu', input_shape=(train.shape[1], ), kernel_initializer='random_normal'))
model_log.add(layers.Dropout(drop))
model_log.add(layers.Dense(256, activation='relu'))
model_log.add(layers.Dense(256, activation='relu'))
model_log.add(layers.Dense(256, activation='relu'))
model_log.add(layers.Dropout(drop))
model_log.add(layers.Dense(256, activation='relu'))
model_log.add(layers.Dense(256, activation='relu'))
model_log.add(layers.Dense(256, activation='relu'))
model_log.add(layers.Dense(256, activation='relu'))
model_log.add(layers.Dense(256, activation='relu'))
model_log.add(layers.Dense(241, activation=(tf.nn.softmax)))

model_log.compile(loss=losses.categorical_crossentropy, 
               optimizer=optimizers.Adam(lr=1e-3), 
               metrics=['categorical_accuracy'])

In [ ]:
model_log.summary()

In [ ]:
history_log = model_log.fit(train_normalized, 
                      train_label_log, 
                      epochs=200, 
                      validation_split=0.2, 
                      batch_size=64, 
                      callbacks=[learning_rate, early_stop])

In [ ]:
prediction_log = model_log.predict(test_normalized)

In [ ]:
fig = plt.figure()
plt.plot(history_lin.history['mean_absolute_error'])
plt.plot(history_lin.history['val_mean_absolute_error'])
plt.title("Średni błąd bezwzględny modelu liniowego")
plt.ylabel("Średni błąd bezwzględny")
plt.xlabel("Epoka")
plt.legend(['Zbiór treningowy', 'Zbiór walidacyjny'], loc='upper left')
plt.show()
# fig.savefig('mae-lin.png')

In [ ]:
fig = plt.figure()
plt.plot(history_lin.history['loss'])
plt.plot(history_lin.history['val_loss'])
plt.title("Funkcja straty modelu liniowego")
plt.ylabel("Wartość straty")
plt.xlabel("Epoka")
plt.legend(['Zbiór treningowy', 'Zbiór walidacyjny'], loc='upper left')
plt.show()
# fig.savefig('loss-lin.png')

In [ ]:
fig = plt.figure()
plt.plot(history_log.history['categorical_accuracy'])
plt.plot(history_log.history['val_categorical_accuracy'])
plt.title("Skuteczność predykcji modelu logistycznego")
plt.ylabel("Skuteczność")
plt.xlabel("Epoka")
plt.legend(['Zbiór treningowy', 'Zbiór walidacyjny'], loc='upper left')
plt.show()
# fig.savefig('accuracy-log.png')

In [ ]:
fig = plt.figure()
plt.plot(history_log.history['loss'])
plt.plot(history_log.history['val_loss'])
plt.title("Funkcja straty modelu logistycznego")
plt.ylabel("Wartość straty")
plt.xlabel("Epoka")
plt.legend(['Zbiór treningowy', 'Zbiór walidacyjny'], loc='upper left')
plt.show()
# fig.savefig('loss-log.png')

In [ ]:
results_lin = model_lin.evaluate(test_normalized, test_label_lin)
print(('Średni błąd bezwzględny wynosi wynosi: %2.2f')%(results_lin[1]))
print(('Przewidywany NMaxPoint średnio odbiega od wartości docelowych właśnie o: %2.2f punkta')%(results_lin[1]))

In [ ]:
results_log = model_log.evaluate(test_normalized, test_label_log)
print(('Dokladnosc zbioru testowego wynosi wynosi: %2.2f%s')%(results_log[1]*100, '%'))

In [ ]:
prediction_lin = model_lin.predict(test_normalized)
prediction_log = model_log.predict(test_normalized)

In [ ]:
x = test_label_lin
y_lin = np.round(prediction_lin).astype('int64')
y_lin = y_lin.reshape(y_lin.shape[0])
z_lin = x - y_lin

In [ ]:
x = test_label_log.argmax(axis=1)
y_log = prediction_log.argmax(axis=1)
z_log = x - y_log

W sytuacji idealnej spodziewamy się zestawu punktów układających się w linię prostą.

In [ ]:
fig = plt.figure()
hist, xbins, ybins, im = plt.hist2d(x=x, y=y_lin, bins=100, cmap='jet', norm = LogNorm())
plt.xlabel('N')
plt.ylabel('N oszacowany')
plt.show()
# fig.savefig('Macierz korelacji - lin.png')

In [ ]:
fig = plt.figure()
hist, xbins, ybins, im = plt.hist2d(x=x, y=y_log, bins=100, cmap='jet', norm = LogNorm())
plt.xlabel('N')
plt.ylabel('N oszacowany')
plt.show()
# fig.savefig('Macierz korelacji - log.png')

W sytuacji idealnej spodziewamy się jednego słupka na pozycji zero:

In [ ]:
fig = plt.figure()
y_lin, x, patches = plt.hist(z_lin, bins = 100)
plt.yscale('log')
plt.xlabel('N - N oszacowany')
plt.grid()
plt.axvline(5, color='k', linestyle='dashed', linewidth=1)
plt.axvline(-5, color='k', linestyle='dashed', linewidth=1)
plt.show()
# fig.savefig('Rzeczywisteaoszacowane-lin.png')

In [ ]:
fig = plt.figure()
y_log, x, patches = plt.hist(z_log, bins = 100)
plt.yscale('log')
plt.xlabel('N - N oszacowany')
plt.grid()
plt.axvline(5, color='k', linestyle='dashed', linewidth=1)
plt.axvline(-5, color='k', linestyle='dashed', linewidth=1)
plt.show()
# fig.savefig('Rzeczywisteaoszacowane-log.png')